In [1]:
from sklearn.datasets import make_classification

# Generate a 2-class classification dataset with 5 features and 200 instances
X, y = make_classification(n_samples=2000, n_features=5, n_informative=2, n_redundant=0, n_clusters_per_class=1, n_classes=2, random_state=0)

In [2]:
from AITIA.uncertainty_system import UncertaintyEstimator

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [4]:
model = LogisticRegression(random_state=0)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
us = UncertaintyEstimator(model=model, verbose=True)
us.fit(X_train, y_train)
y_pred, misclassification_risk = us.predict(X_test)